<hr style="border:2px solid gray"> </hr>

# Homework 1 - Traveling Salesman Problem

## Example Code

### Algorithm 4: Genetic Algorithm 

### Author: Wangduk Seo (CAU AI Lab)
<hr style="border:2px solid gray"> </hr>

# Step 0. Importing packages and Global Settings

---------------------------------------------------------------
## (Optional) For Colab

In [ ]:
# from google.colab import drive
# import os, sys
# drive.mount('gdrive', force_remount=True)

---------------------------------------------------------------

In [ ]:
# package list
import numpy as np
import sys
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import time

# Global Variables
# GA
POOL_SIZE = 4 
TOURNAMENT_SIZE = 3
MAX_ITERATION = 1000

# SA
MAX_EVALUATION = 10
SUB_ITERATIONS = 5
TEMPERATURE = 100
COOLING_RATIO = 0.4
TEMP_LIMIT = 1

np.random.seed(0)
# Plot Settings
PLOT_MODE = True # Draw Route
plt.ion()

# First City Index
FIRST_IDX = 0

In [ ]:
file_path = 'data1.txt'

# Step 1. Data Loading

In [ ]:
def fileloader():
    #     Data Format
    #     ---------------------------------------------------------
    #     NAME : pia3056
    #     COMMENT : Bonn VLSI data set with 3056 points
    #     COMMENT : Uni Bonn, Research Institute for Discrete Math
    #     COMMENT : Contributed by Andre Rohe
    #     TYPE : TSP
    #     DIMENSION : 3056 -----------------------------|
    #     EDGE_WEIGHT_TYPE : EUC_2D                     |
    #     NODE_COORD_SECTION                            |
    #     1 0 11 (2 dimentional coordinate of city)     |
    #     2 0 115                                       |
    #     ...                                           |
    #     ...(Total 3056 nodes)<------------------------|
    #     EOF
    #     ---------------------------------------------------------
    with open(file_path, "r") as file:
        file_str = file.readlines()

    # Get the coordinates of cities
    coord_str = file_str[7:-1]  # first city string to last city string (EOF 전까지)
    coord_list = np.zeros((len(coord_str), 2))
    for idx, item in enumerate(coord_str):
        items = item.split()
        coord_list[idx, 0], coord_list[idx, 1] = int(items[1]), int(items[2])

    return coord_list

# Step 2. Initialization

In [ ]:
def initialize_greedy(coord_list, first_idx):
    cnt_cities = len(coord_list)
    # Initialize path and insert first city index to the first and last elements
    path = np.zeros(cnt_cities + 1, dtype=np.int32)
    path[0], path[-1] = first_idx, first_idx

    # Euclidean distance map between cities
    path_map = euclidean_distances(coord_list, coord_list)

    cities_tovisit = np.ones((cnt_cities), dtype=np.bool_)
    cities_tovisit[first_idx] = False

    # Iteratively Connect nearest cities
    for i in range(1, cnt_cities):
        start_idx = path[i - 1]
        distance_from_start = path_map[start_idx, :]
        nearest_list = np.argsort(distance_from_start)
        for idx in range(len(nearest_list)):
            # check the nearest city is visited
            if cities_tovisit[nearest_list[idx]]:
                nearest_city = nearest_list[idx]
                break
        cities_tovisit[nearest_city] = False
        path[i] = nearest_city

    return path_map, path

def initialize_greedy_improve(coord_list, first_idx):
    cnt_cities = len(coord_list)
    # Initialize path and insert first city index to the first and last elements
    path = np.zeros(cnt_cities + 1, dtype=np.int32)
    path[0], path[-1] = FIRST_IDX,FIRST_IDX  

    # Euclidean distance map between cities
    path_map = euclidean_distances(coord_list, coord_list)

    cities_tovisit = np.ones((cnt_cities), dtype=np.bool_)
    cities_tovisit[FIRST_IDX] = False
    cities_divide=np.zeros((cnt_cities),dtype=np.int32)

    path_map_copy=path_map.copy()
    longest_from_start=np.argsort(-path_map_copy[0])

  

    slope=(coord_list[longest_from_start[0],1]-coord_list[0,1])/(coord_list[longest_from_start[0],0]-coord_list[0,0])
    count_1=0
    count_2=0
    for i in range(0,cnt_cities):
        if coord_list[i,1]>=slope*(coord_list[i,0]-coord_list[0,0])+coord_list[0,1] and count_1<cnt_cities/2:
            cities_divide[i]=1
            count_1=count_1+1
        elif count_2<cnt_cities/2:
            cities_divide[i]=2
        else:
            cities_divide[i]=1

    for i in range(1, cnt_cities):
        start_idx = path[i - 1]

        distance_from_start = path_map[start_idx, :]
        nearest_list = np.argsort(distance_from_start)
        nearest_city=-1
        temp=0
        checker=0

        for idx in range(len(nearest_list)):
            # check the nearest city is visited
            if cities_tovisit[nearest_list[idx]]:
                if checker==0:
                    temp=nearest_list[idx]
                    checker=1
                if cities_divide[path[i-1]]==cities_divide[nearest_list[idx]]:
                    nearest_city = nearest_list[idx]
                    
                    break
        if nearest_city==-1:
            # check the nearest city is visited
            nearest_city = temp
            
            
        cities_tovisit[nearest_city] = False
        path[i] = nearest_city

    return path_map, path


def initialize_random(coord_list, first_idx):
    cnt_cities = len(coord_list)
    path = np.zeros(cnt_cities + 1, dtype=np.int32)

    path[0], path[-1] = first_idx, first_idx
    # Euclidean distance map between cities
    path_map = euclidean_distances(coord_list, coord_list)

    # city indices without first city index
    cities_tovisit = np.delete(np.arange(cnt_cities), first_idx)
    cities_random = np.random.permutation(cities_tovisit)
    path[1:-1] = cities_random

    return path_map, path

def path_cost(path_map, path):
    # The array of cost between cities in the path
    cnt_cities = path_map.shape[0]
    cost_arr = np.zeros(cnt_cities)
    for i in range(cnt_cities):
        cost_arr[i] = path_map[path[i], path[i+1]]

    return cost_arr

# Step 3. Searching a path

## Algorithm 4. Genetic Algorithm 

In [ ]:
def two_opt_swap(path_map, path, iterations, coord_list):
    cnt_cities = path_map.shape[0]
    # Save the best path

    cost_arr = path_cost(path_map, path)
    best_path = path.copy()
    best_cost = cost_arr.sum()
    
    for i in range(iterations):
        curr_path = best_path.copy()
        # Select two indices of flip points
        if i%4==0:
            sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities + 1), 2))

        elif i%4==1:
            cost_idx = np.argsort(-cost_arr)
            sel_idx[0] = curr_path[cost_idx[0]]+1
            sel_idx[1] = curr_path[cost_idx[int(cnt_cities/2)]]+1
            sel_idx.sort()
        elif i%4==2:
            cost_idx = np.argsort(-cost_arr)
            sel_idx[0] = curr_path[cost_idx[0]]+1
            sel_idx[1] = (curr_path[cost_idx[0]+1]+curr_path[cost_idx[0]])/2+1
            sel_idx.sort()
        else:
            cost_idx = np.argsort(-cost_arr)
            sel_idx[0] = curr_path[cost_idx[0]]+1
            sel_idx[1]=np.random.choice(np.arange(1, cnt_cities + 1), 1)
            sel_idx.sort()

        # Path Flip and update cost array
        curr_path[sel_idx[0]:sel_idx[1]] = np.flip(curr_path[sel_idx[0]: sel_idx[1]])
        cost_arr = path_cost(path_map, curr_path)

        # Compare to the best path
        curr_cost = cost_arr.sum()
        if curr_cost < best_cost:
            best_path = curr_path
            best_cost = curr_cost
    
    temperature = TEMPERATURE
    i=0
    checker=0
    global COOLING_RATIO
    while temperature > TEMP_LIMIT:
        curr_path = best_path.copy()
        # Select two indices of flip points

        if i%4==0:
            sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities + 1), 2))

        elif i%4==1:
            cost_idx = np.argsort(-cost_arr)
            sel_idx[0] = curr_path[cost_idx[0]]+1
            sel_idx[1] = curr_path[cost_idx[int(cnt_cities/2)]]+1
            sel_idx.sort()
        elif i%4==2:
            cost_idx = np.argsort(-cost_arr)
            sel_idx[0] = curr_path[cost_idx[0]]+1
            sel_idx[1] = (curr_path[cost_idx[0]+1]+curr_path[cost_idx[0]])/2+1
            sel_idx.sort()
        else:
            cost_idx = np.argsort(-cost_arr)
            sel_idx[0] = curr_path[cost_idx[0]]+1
            sel_idx[1]=np.random.choice(np.arange(1, cnt_cities + 1), 1)
            sel_idx.sort()
        i=i+1
        # Path Flip and update cost array
        curr_path[sel_idx[0]:sel_idx[1]] = np.flip(curr_path[sel_idx[0]: sel_idx[1]])
        cost_arr = path_cost(path_map, curr_path)
        curr_cost = cost_arr.sum()
        if checker==1:
            COOLING_RATIO = COOLING_RATIO*1.1
            checker=2
        if curr_cost <= best_cost:
            best_path, best_cost = curr_path, curr_cost
            if checker==2:
                COOLING_RATIO = COOLING_RATIO*0.8
                temperature = temperature * COOLING_RATIO
        else:
            prob = 1 / np.exp((curr_cost - best_cost) / float(temperature))
            if prob > np.random.rand(1):
                best_path, best_cost = curr_path, curr_cost
                COOLING_RATIO=COOLING_RATIO*0.7
                temperature = temperature * COOLING_RATIO
                checker=1
            if i%5==4:
                temperature=temperature*COOLING_RATIO
        
    return best_path, best_cost

In [ ]:
def sa(path_map, path, coord_list):
    best_path, best_cost = path.copy() , path_cost(path_map, path).sum()

    for i in range(MAX_EVALUATION):
        curr_path = best_path.copy()
        new_path, new_cost = two_opt_swap(path_map, curr_path, SUB_ITERATIONS, coord_list)

        if new_cost < best_cost:
            best_path, best_cost = new_path, new_cost
            
    return best_path, best_cost

In [ ]:
def initialization(coord_list):
    cnt_cities = len(coord_list)
    path_pool = np.zeros((POOL_SIZE, cnt_cities + 1), dtype=np.int32)
    pool_cost = np.zeros(POOL_SIZE)
    
    path_map, path_pool[0, :] = initialize_greedy(coord_list, FIRST_IDX)
    pool_cost[0] = path_cost(path_map, path_pool[0, :]).sum()

    print('Path {} is initialized'.format(0))
    
    _, path_pool[1, :] = initialize_greedy_improve(coord_list, FIRST_IDX)
    pool_cost[1] = path_cost(path_map, path_pool[1, :]).sum()
    print('Path {} is initialized'.format(1))

    for i in range(2, POOL_SIZE):
        _, path_pool[i, :] = initialize_random(coord_list, FIRST_IDX)
        path_pool[i, :], pool_cost[i] = sa(path_map, path_pool[i, :], coord_list)
        print('Path {} is initialized'.format(i))
    
    return path_pool, pool_cost, path_map

In [ ]:
def selection(pool_cost,i, TOURNAMENT_SIZE, sel_size=2):
    # tournament selection
    if i%3==0:
        sel_idx = np.random.choice(POOL_SIZE, TOURNAMENT_SIZE, replace=False)
        sel_cost = pool_cost[sel_idx]
        best_idx = sel_idx[np.argsort(sel_cost)][:sel_size]
        
    elif i%3==1:
        idx_sort=np.argsort(pool_cost)
        sel_idx=idx_sort
        sel_cost = pool_cost[sel_idx]
        best_idx = sel_idx[np.argsort(sel_cost)][:sel_size]
    else:
        idx_sort=np.argsort(pool_cost)
        sel_idx=np.zeros(2,dtype=np.int32)
        sel_idx[0]=idx_sort[0]
        sel_idx[1]=np.random.choice(POOL_SIZE,1)
        while sel_idx[0]==sel_idx[1]:
            sel_idx[1]=np.random.choice(POOL_SIZE,1)
        best_idx=sel_idx
    return best_idx

In [ ]:
# pmx crossover
def crossover(path1, path2):
    cnt_cities = len(path1) - 1
    # Select two indices of crossover points
    sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))
    # Initialize child path
    child_path = np.zeros(cnt_cities + 1, dtype=np.int32)
    child_path[0], child_path[-1] = -1, -1
    # Copy the path between crossover points
    child_path[sel_idx[0]:sel_idx[1]] = path1[sel_idx[0]:sel_idx[1]]
    # Copy the rest of the path from path2
    path2_idx = np.where(np.isin(path2, child_path) == False)[0]
    child_path[np.where(child_path == 0)[0]] = path2[path2_idx]
    child_path[0], child_path[-1] = FIRST_IDX, FIRST_IDX

    return child_path


In [ ]:
# swap mutation
def mutation(path):
    cnt_cities = len(path)
    child_path = path.copy()
    # Select two indices of mutation points
    sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))
    # Swap the path between mutation points
    child_path[sel_idx[0]:sel_idx[1]] = np.flip(child_path[sel_idx[0]:sel_idx[1]])

    return child_path 


In [ ]:
# genetic algorithm
def ga(coord_list):
    best_cost = np.Inf
    print('Start Genetic Algorithm')
    print('Initialize the population')
    path_pool, pool_cost, path_map = initialization(coord_list)
    print('Start the evolution')
    for i in range(MAX_ITERATION):
        if (i+1) % 1000 == 0:
            print('Iteration {}'.format(i + 1))
        # selection
        sel_idx = selection(pool_cost,i, TOURNAMENT_SIZE)
        # crossover
        child_crx_1 = crossover(path_pool[sel_idx[0]], path_pool[sel_idx[1]])
        cost_crx_1 = path_cost(path_map, child_crx_1).sum()
        child_crx_2 = crossover(path_pool[sel_idx[1]], path_pool[sel_idx[0]])
        cost_crx_2 = path_cost(path_map, child_crx_2).sum()
        if cost_crx_1>cost_crx_2:
            child_crx=child_crx_2
            cost_crx=cost_crx_2
        else:
            child_crx=child_crx_1
            cost_crx=cost_crx_1
        
        # mutation
        sel_idx = selection(pool_cost,i, TOURNAMENT_SIZE, sel_size=1)
        child_mut = mutation(path_pool[sel_idx[0]])
        cost_mut = path_cost(path_map, child_mut).sum()
        # replace
        sort_idx = np.argsort(pool_cost)

        path_pool[sort_idx[-1]], pool_cost[sort_idx[-1]] = child_crx, cost_crx 
        path_pool[sort_idx[-2]], pool_cost[sort_idx[-2]] = child_mut, cost_mut 

        cur_idx = np.argmin(pool_cost)
        cur_path = path_pool[cur_idx]
        cur_cost = pool_cost[cur_idx]

        if best_cost > cur_cost:
            best_cost = cur_cost
            if PLOT_MODE:
                plt.close()
                figure, ax = plt.subplots()
                plt.scatter(coord_list[:, 0], coord_list[:, 1], c='red', s=10)
                plt.title('City Route: Iteration {}'.format(i + 1))
                coord_path = coord_list
                coord_path = np.append(coord_path, coord_path[FIRST_IDX, :].reshape(1, 2), axis=0)
                coord_path[:, :] = coord_path[cur_path, :]
                lines, = ax.plot(coord_path[:, 0], coord_path[:, 1], 'k--')
                figure.canvas.draw()
                figure.canvas.flush_events()
                plt.show()

    best_idx = np.argmin(pool_cost)
    return path_pool[best_idx], pool_cost[best_idx]

# Main

In [ ]:
# Step 1
try:
    coord_list = fileloader()
except Exception as e:
    print('예외 발생', e)
    sys.exit()

start_time = time.time()

best_path, best_cost = ga(coord_list)

print('Execution Time: ' + str(time.time() - start_time))
print('Path: ' + str(best_path.tolist()))
print('Cost: ' + str(best_cost))

In [ ]:
initialization단계에서 수정한 코드와 기존 greedy코드를 함께 사용
selection에서 랜덤한 두 case 선택
가장 우수한 두 case 선택
우수한 한 case와 random한 case선택